In [225]:
import pandas as pd
import glob
import numpy as np
from datetime import datetime
import datetime as dt
import csv

now = datetime.now()
format = "%m%d%Y"
time1 = now.strftime(format)

print("Import complete")

Import complete


In [226]:
def Grouper(df,column,criteria):
    column = column
    grouped = df.groupby(df[column])
    g = grouped.get_group(criteria)
    return g

def InferName(folder,doc):
    files = [file.split("\\")[-1].split(".")[0] for file in glob.glob(folder)]
    char = ["_","-","/"]
    splitter = "None"
    for c in char:
        countl = [f for f in files if c in f]
        if len(countl) == len(files):
            splitter = c
    if splitter == "None":
        return doc, splitter
    else:
        ind = "None"
        flist = [f.split(splitter) for f in files]
        for i in range(0,len(flist[0])):
            ilist = list(set([f[i] for f in flist]))
            if len(ilist) == len(flist):
                ind = i
        if ind == "None":
            return doc
        else:
            dname = doc.split("\\")[-1].split(".")[0].split(splitter)[ind]
            return dname

In [227]:
def SplitSource(file):
    with open(file) as f:
        reader = csv.reader(f)
        S = [row for row in reader]
    user = []
    master = []
    for line in S:
        user.append(line)
        if '</user>' in line[0]:
            master.append(user)
            user = []
    return master

def GetDomain(file):
    domain = file.split("\\")[-1].split("_")[0]
    return domain

def ChooseGroups(users):
    groups = [entry for entry in users if 'user group="" admin="false" dn=":GROUP:' in entry[0][0]]
    fullperms = []
    for g in groups:
        groupname = g[0][0].split(':')[-1].replace('">','')
        g.pop()
        perms = []
        for line in g[2:]:
            try:
                perm = line[0].split('name=')[-1].split('<')[0].split(">")
                name = perm[0].split('"')[1]
                level = perm[-1]
                perms.append([name,level])
            except:
                print("Error with:",line)
        fullperms.append([groupname,perms])
    return fullperms
        
def CreateMasterList(users):
    fullperms = ChooseGroups(users)
    groups = [g[0] for g in fullperms]
    data = [g[1] for g in fullperms]
    unames = []
    for d in data:
        for item in d:
            if item[0] not in unames:
                unames.append(item[0])
    masterlist = []
    for n in unames:
        pg = [n]
        for d in data:
            names = [item[0] for item in d]
            if n not in names:
                pg.append('0 - None')
            else:
                for line in d:
                    if line[0] == n:
                        if line[1] == '0':
                            pg.append("0 - None")
                        elif line[1] == '1':
                            pg.append("1 - View")
                        elif line[1] == '2':
                            pg.append("2 - View/Edit")
                        elif line[1] == '3':
                            pg.append("3 - View/Edit/Add/Delete")
                        else:
                            pg.append(line[1])
        masterlist.append(pg)
    return masterlist, groups

def GetPermsMatrix(file):
    domain = GetDomain(file)
    users = SplitSource(file)
    ML, groups = CreateMasterList(users)
    columns = ["Product","Category","Permission"] + groups
    master = []
    for line in ML:
        names = line.pop(0)
        PCP = names.split("/")[1:]
        newline = PCP+line
        master.append(newline)
    hrdf = pd.DataFrame(master,columns=columns)
    return hrdf, domain

print("Module Loaded")

Module Loaded


In [228]:
with pd.ExcelWriter('Reports/1.HumanReadablePermissions.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            hrdf, domain = GetPermsMatrix(file)
            hrdf.to_excel(writer, sheet_name=domain,index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


In [229]:
def GetCompareMatrix(file):
    comparedf, comparedomain = GetPermsMatrix(file)
    ind = ["Product","Category","Permission"]
    groups = [col for col in masterdf.columns if col not in ind]
    m = []
    for col in groups:
        try:
            newcol = str(col+" - Update")
            comparedf[newcol] = comparedf[col]
            comparedfa = comparedf[ind + [newcol]]
            masterdfa = masterdf[ind + [col]]
            matrix = masterdfa.merge(comparedfa, on=ind)
            matrix['Update'] = np.where(matrix[col] != matrix[newcol],0,1)
            matrix = Grouper(matrix,'Update',0)
            matrix = matrix[ind + [col]]
            m.append(matrix)
        except:
            pass

    mast = masterdf[ind]
    x = 1
    while x < len(m):
        c = m[x]
        c.reset_index()
        mast = mast.merge(c,how='outer')
        x += 1
    permcols = [col for col in mast.columns if col not in ind]
    mast = mast.dropna(how='all',subset=permcols)
    return mast

print("Module Loaded")

Module Loaded


In [230]:
md = 'peapp1'

for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1] and GetDomain(file) == md:
            masterdf, domain = GetPermsMatrix(file)
with pd.ExcelWriter('Reports/2.ComparePermissionsMatrix.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            comparedf, comparedomain = GetPermsMatrix(file)
            mast = GetCompareMatrix(file)
            mast.to_excel(writer, sheet_name=comparedomain,index=False)

print("Permissions Loaded to File")

Permissions Loaded to File


In [231]:
def GetUsers(file):
    domain = GetDomain(file)
    groups = SplitSource(file)
    users = [entry for entry in groups if 'user group="|USR|' in entry[0][0]]
    master = []
    for entry in users:
        u = ['User']
        name = "None"
        for line in entry:
            if 'Full Name' in line[0]:
                name = line[0].split(">")[1].split("<")[0]
        u.append(name)
        roles = []
        header = entry[0]
        if len(header) == 1:
            al = header[0].split(" ")
        else:
            roles.append(header[0].split('"')[-1])
            x = 3
            while x <= len(header):
                roles.append(header[x-2])
                x += 1
            al = header[-1].split(" ")
        for item in al:
            if 'dn=' in item:
                un = item.split('"')[1]
                u.append(un)
        if 'admin="true"' in al:
            u.append('Yes')
        else:
            u.append("No")
        for r in al:
            if '|USR|' in r:
                rs = [item for item in r.split('"') if '|USR|' in item]
                for item in rs:
                    roles.append(item)
        u.append(roles)
        master.append(u)
        
    consoles = [entry for entry in groups if 'user group="|CON|' in entry[0][0]]
    for entry in consoles:
        u = ['Console']
        name = "None"
        for line in entry:
            if 'Description' in line[0]:
                name = line[0].split(">")[1].split("<")[0]
        u.append(name)
        roles = []
        header = entry[0]
        if len(header) == 1:
            al = header[0].split(" ")
        else:
            roles.append(header[0].split('"')[-1])
            x = 3
            while x <= len(header):
                roles.append(header[x-2])
                x += 1
            al = header[-1].split(" ")
        for item in al:
            if 'dn=' in item:
                un = item.split(':')[-1].split('"')[0]
                u.append(un)
        if 'admin="true"' in al:
            u.append('Yes')
        else:
            u.append("No")
        for r in al:
            if '|CON|' in r:
                rs = [item for item in r.split('"') if '|CON|' in item]
                for item in rs:
                    roles.append(item)
        u.append(roles)
        master.append(u)
    df = pd.DataFrame(master,columns=['Type','Assigned To','Name','Admin?','Groups'])
                
                    
    return df, domain

def ConsoleUserRole(file):
    df, domain = GetUsers(file)
    user = Grouper(df,'Type','User')
    console = Grouper(df,'Type','Console')
    console = console[console['Assigned To'] != '']
    cur = console.merge(user,on=['Assigned To'],how='inner')
    return cur, domain
        
print("Module Loaded")

Module Loaded


In [232]:
with pd.ExcelWriter('Reports/3.UsersAndConsoles.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            df, domain = GetUsers(file)
            df.to_excel(writer, sheet_name=domain,index=False)

with pd.ExcelWriter('Reports/4.UsersAndConsoles - All Domains.xlsx') as writer:
    masterlist = []
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            df, domain = GetUsers(file)
            df['Domain'] = domain
            masterlist.append(df)
    df = pd.concat(masterlist)
    data = df.values.tolist()
    N = list(set([item[2] for item in data]))
    ml = []
    for n in N:
        alist = list(set([line[1] for line in data if line[2] == n]))
        tlist = list(set([line[0] for line in data if line[2] == n]))
        dlist = list(set([line[5] for line in data if line[2] == n]))
        glist = []
        for line in data:
            if line[2] == n:
                for item in line[4]:
                    if item not in glist:
                        glist.append(item)
                
        ml.append([n,alist,tlist,dlist,glist])
    df = pd.DataFrame(ml,columns=['Name','Assigned To','Type','Domains','Groups'])
    df.to_excel(writer, sheet_name="All",index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


In [233]:
masterlist = []
for file in glob.glob('uds/all_UDS/*'):
    if "uds.xml" in file.split("\\")[-1]:
        df, domain = GetUsers(file)
        df['Domain'] = domain
        masterlist.append(df)
df = pd.concat(masterlist)
print(df.shape)

(13718, 6)


In [234]:
#print(df.columns)
#data = df.values.tolist()
#print(len(data))
#N = list(set([item[2] for item in data]))
#l = []
#for n in N:
    #alist = list(set([line[1] for line in data if line[2] == n]))
    #tlist = list(set([line[0] for line in data if line[2] == n]))
    #dlist = list(set([line[5] for line in data if line[2] == n]))
    #ml.append([n,alist,tlist,dlist])
#df = pd.DataFrame(master,columns=['Name','Assigned To','Type','Domains'])


In [235]:
with pd.ExcelWriter('Reports/5.ConsoleUserRole.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            cur, domain = ConsoleUserRole(file)
            cur.to_excel(writer, sheet_name=domain,index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


In [236]:
def GetRoleDomainMatrix(file):
    domain = GetDomain(file)
    users = SplitSource(file)
    groups = [entry for entry in users if 'user group="" admin="false" dn=":GROUP:' in entry[0][0]]
    gnames = [g[0][0].split(':')[-1].replace('">','') for g in groups]
    return gnames, domain


allgroups = []
with pd.ExcelWriter('Reports/6.RoleDomainMatrix.xlsx') as writer:
    for file in glob.glob('uds/uds_setdat/*'):
        if "uds.xml" in file.split("\\")[-1]:
            groups, domain = GetRoleDomainMatrix(file)
            for g in groups:
                if g not in allgroups:
                    allgroups.append(g)
    allgroups = sorted(allgroups)
    df = pd.DataFrame(allgroups,columns=['Group'])
    for file in glob.glob('uds/uds_setdat/*'):
        if "uds.xml" in file.split("\\")[-1]:
            glist = []
            groups, domain = GetRoleDomainMatrix(file)
            for g in allgroups:
                if g in groups:
                    glist.append("Yes")
                else:
                    glist.append("No")
            df[domain] = glist
                
    df.to_excel(writer, sheet_name=domain,index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


## User Domain Matrix

In [237]:
def GetUserDomainMatrix(file):
    df, domain = GetUsers(file)
    user = Grouper(df,'Type','User')
    return user, domain


allusers = []
with pd.ExcelWriter('Reports/7.UserDomainMatrix.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Assigned To'] +" - " + user['Name']
            ulist = user['UName'].tolist()
            for u in ulist:
                if u not in allusers:
                    allusers.append(u)
    allusers = sorted(allusers)
    df = pd.DataFrame(allusers,columns=['User'])
    for file in glob.glob('uds/uds_setdat/*'):
        if "uds.xml" in file.split("\\")[-1]:
            ulist = []
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Assigned To'] +" - " + user['Name']
            for u in allusers:
                groups = user.loc[user['UName'] == u, 'Groups'].tolist()
                if groups != []:
                    ulist.append(groups)
                else:
                    ulist.append("")
                    

            df[domain] = ulist
                
    df.to_excel(writer, sheet_name="UserDomainMatrix",index=False)
    
    allusers = []
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Name'] +" - " + user['Assigned To']
            ulist = user['UName'].tolist()
            for u in ulist:
                if u not in allusers:
                    allusers.append(u)
    allusers = sorted(allusers)
    df = pd.DataFrame(allusers,columns=['User'])
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            ulist = []
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Name'] +" - " + user['Assigned To']
            for u in allusers:
                groups = user.loc[user['UName'] == u, 'Groups'].tolist()
                if groups != []:
                    ulist.append(groups)
                else:
                    ulist.append("")
                    

            df[domain] = ulist
                
    df.to_excel(writer, sheet_name="UserDomainMatrix - Inverse",index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


## Console Domain Matrix

In [238]:
def GetUserDomainMatrix(file):
    df, domain = GetUsers(file)
    user = Grouper(df,'Type','Console')
    return user, domain


allusers = []
with pd.ExcelWriter('Reports/8.ConsoleDomainMatrix.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Assigned To'] +" - " + user['Name']
            ulist = user['UName'].tolist()
            for u in ulist:
                if u not in allusers:
                    allusers.append(u)
    allusers = sorted(allusers)
    df = pd.DataFrame(allusers,columns=['Console'])
    for file in glob.glob('uds/uds_setdat/*'):
        if "uds.xml" in file.split("\\")[-1]:
            ulist = []
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Assigned To'] +" - " + user['Name']
            for u in allusers:
                groups = user.loc[user['UName'] == u, 'Groups'].tolist()
                if groups != []:
                    ulist.append(groups)
                else:
                    ulist.append("")
                    

            df[domain] = ulist
                
    df.to_excel(writer, sheet_name="ConsoleDomainMatrix",index=False)
    
    allusers = []
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Name'] +" - " + user['Assigned To']
            ulist = user['UName'].tolist()
            for u in ulist:
                if u not in allusers:
                    allusers.append(u)
    allusers = sorted(allusers)
    df = pd.DataFrame(allusers,columns=['Console'])
    for file in glob.glob('uds/uds_setdat/*'):
        if "uds.xml" in file.split("\\")[-1]:
            ulist = []
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Name'] +" - " + user['Assigned To']
            for u in allusers:
                groups = user.loc[user['UName'] == u, 'Groups'].tolist()
                if groups != []:
                    ulist.append(groups)
                else:
                    ulist.append("")
                    

            df[domain] = ulist
                
    df.to_excel(writer, sheet_name="ConsoleDomainMatrix - Inverse",index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


In [239]:
ag = [g for g in allgroups if 'CON' not in g]
with pd.ExcelWriter('Reports/9.UserDomainMatrix - by Role.xlsx') as writer:
    for gr in ag:
        g = gr.split("|")[-1].split("_")[0]
        allusers = []
        for file in glob.glob('uds/all_UDS/*'):
            if "uds.xml" in file.split("\\")[-1]:
                userraw, domain = GetUserDomainMatrix(file)
                user = userraw[userraw['Groups'].astype(str).str.contains(g)==True]
                user['UName'] = user['Assigned To'] +" - " + user['Name']
                ulist = user['UName'].tolist()
                for u in ulist:
                    if u not in allusers:
                        allusers.append(u)
        allusers = sorted(allusers)
        df = pd.DataFrame(allusers,columns=['User'])
        for file in glob.glob('uds/all_UDS/*'):
            if "uds.xml" in file.split("\\")[-1]:
                ulist = []
                userraw, domain = GetUserDomainMatrix(file)
                user = userraw[userraw['Groups'].astype(str).str.contains(g)]
                user['UName'] = user['Assigned To'] +" - " + user['Name']
                for u in allusers:
                    groups = user.loc[user['UName'] == u, 'Groups'].tolist()
                    if groups != []:
                        ulist.append(groups)
                    else:
                        ulist.append("")
                    

                df[domain] = ulist
                
        try:
            df.to_excel(writer, sheet_name=gr,index=False)
        except:
            pass
            
print("Permissions Loaded to File")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Permissions Loaded to File


In [314]:
def GetCIPAccess(file):
    with open(file) as f:
        reader = csv.reader(f)
        S = [row for row in reader]
        R = list(set([item[0] for item in S]))
        access = list(set([item[2] for item in S]))
        mlist = []
        for r in R:
            nlist = list(set([item[1] for item in S if item[0] == r]))
            alist = list(set([item[2] for item in S if item[0] == r]))
            mlist.append([r,nlist,alist])
    return mlist, access
        
directory = 'PAM\*'
Target = max([InferName(directory,f) for f in glob.glob(directory) if 'AllUser' in f])
for file in glob.glob(directory):
    domain = InferName(directory, file)
    if domain == Target:
        CIPA, access = GetCIPAccess(file)

Domain_Dict = {
    'TMS':['prod31521','prod3821','prod','ldan1','pdac1','bdac2','edan1','edan2','tdan2','wdan1','ldan2','bdac1','pdac2','tdan1','wdan2'],
    'QAS':['pqdac2','eqasdan1','tqasdan1','wqasdan2', 'bqdac2','pqstudy2','pqdac1', 'wqasdan1','bqdac1','tqasdan2','lqasdan1',
           'eqasdan2','lqasdan2'],
    'ENG':['peapp1'],
    'OTS':['pots2','pots1'],
    'PDS':['epdan2','wpdan1', 'lpdan2','lpdan1','wpdan2','epdan1','tpdan2','tpdan1','ppds1','bpds1'],
    'REMOTE':['tstudy1','wstudy1','pqstudy1','pstudy1','bstudy1','estudy1','bstudy2','pstudy2','bqstudy1','tqasstudy1',
              'lstudy1', 'bqstudy2']
}

server_dict = {}
directory = 'uds/all_UDS/*'
for file in glob.glob('uds/all_UDS/*'):
    server = InferName(directory,file)
    domain = "None"
    for key, value in Domain_Dict.items():
        if server in value:
            domain = key
    server_dict.update({server:domain})

OSI_U = []
for file in glob.glob('uds/all_UDS/*'):
    df, server = GetUsers(file)
    df['Server'] = server
    df['Domain'] = server_dict.get(server)
    OSI = df.values.tolist()
    OSI_Users = [line for line in OSI if line[0]=='User']
    for line in OSI_Users:
        OSI_U.append(line)

file = 'Analysis/PAM Access Match.csv'
with open(file) as f:
    reader = csv.reader(f)
    Match = [row for row in reader]
    RACF = [m[1] for m in Match]
    UN = [m[0] for m in Match]

print("Users that Need to be Added to the Match File")
for line in OSI_U:
    r = [m[1] for m in Match if m[0]==line[2]]
    if len(r) == 0:
        print(line)
print("Validation Complete")

print("Users that have multiple entries in the Match File")
for line in OSI_U:
    r = [m[1] for m in Match if m[0]==line[2]]
    if len(r) > 1:
        print(line)
print("Validation Complete")

######

NFED = []
IU = []
IUNC = []
CIPR = [item[0] for item in CIPA]
for line in OSI_U:
    r = [m[1] for m in Match if m[0]==line[2]]
    if r == []:
        continue
    try:
        line.append(r[0])
    except:
        print(line)
    if r[0] == 'Not found in employee directory':
        NFED.append(line)
    elif r[0] not in CIPR:
        IUNC.append(line)
    else:
        IU.append(line)
        

print("Identified Users:",len(IU))
print("Identified Users, Not CIP:",len(IUNC))
print("Not Found in Employee Directory:",len(NFED))

#####

file = 'Analysis\Approved_Access_03.2021\Approved_Access_03.2021.xlsx'
AL_RbR = []
ALDF = pd.read_excel(file, sheet_name='Feedback',na_values = '')
ALDF['domain'] = 'TMS'
AL = ALDF.values.tolist()

A = list(set([line[4] for line in AL]))

for RACF in A:
    newroles = [line[5] for line in AL if line[4]==RACF]
    newline = [RACF,newroles]
    AL_RbR.append(newline)

for line in IU:
    CA = [c[2] for c in CIPA if c[0]==line[-1]]
    line.append(CA)
    AL = [r[1] for r in AL_RbR if r[0]==line[-2]]
    line.append(AL)



print("Module Loaded")

Users that Need to be Added to the Match File
Validation Complete
Users that have multiple entries in the Match File
Validation Complete
Identified Users: 4889
Identified Users, Not CIP: 228
Not Found in Employee Directory: 443
Module Loaded


In [323]:
TMS_Access = ['Field (ODV)','Database (SCADA)','Integration','TGM Supervisor','State Estimator(TNA)','Planning','DOC Dispatcher'
              'DOC Supervisor','Communications (RTU)','Support','TGM Dispatcher','TGO Supervisor']
siemenssubs = {
        "|USR|Database_(SCADA)": "Database (SCADA)",
        "|USR|Integration": "Integration",
        "|USR|State_Estimator(TNA)": "State Estimator(TNA)",
        "|USR|DOC_Operator": "DOC Dispatcher",
        "|USR|Communications_(RTU)": "Communications (RTU)",
        "|USR|TGM_Operator": "TGM Dispatcher",
        "N/A": "Planning",
        "N/A": "DOC Supervisor",
        "N/A": "Support",
        "N/A": "TGO Supervisor",
        "N/A": "Field (ODV)"
}

['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF','CIPAccess','ApprovedRole']
IUV = []
for u in IU:
    for r in u[4]:
        siemensrole = siemenssubs.get(r,r)
        newline = [item for item in u[0:4]] + [r] + [item for item in u[5:8]]
        if siemensrole in u[8][0]:
            newline.append(siemensrole)
            newline.append("Success")
        else:
            newline.append("Access Not Found")
            newline.append("Failure")
        try:
            if r in u[9][0]:
                newline.append(r)
                newline.append("Success")
            else:
                newline.append("Access Not Found")
                newline.append("Failure")
        except:
            newline.append("Not Originally Approved for Access")
            newline.append("Failure")
        if newline[9] == "Success" and newline[11] == "Success":
            newline.append("None")
        elif newline[11] == "Success" and newline[9] != "Success":
            newline.append("True-up PAM")
        else:
            newline.append("Review")
        IUV.append(newline)


with pd.ExcelWriter('Reports/10.IdentifiedUsers.xlsx') as writer:
    IUDF = pd.DataFrame(IU,columns=['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF','CIPAccess','ApprovedRole'])
    IUDF.to_excel(writer, sheet_name='IdentifiedUsers',index=False)
    IUDF = pd.DataFrame(IUV,columns=['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF','CIPAccess','VC1','ApprovedRole',"VC2","Recommended_Action"])
    IUDF.to_excel(writer, sheet_name='Validation',index=False)
    IUNCDF = pd.DataFrame(IUNC,columns=['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF'])
    IUNCDF.to_excel(writer, sheet_name='NoCIP',index=False)
    NFEDDF = pd.DataFrame(NFED,columns=['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF'])
    NFEDDF.to_excel(writer, sheet_name='NotFoundInEmpDirectory',index=False)
    RoleMapping = pd.DataFrame(list(siemenssubs.items()),columns=['OSI_Role','Siemens_Role'])
    RoleMapping.to_excel(writer, sheet_name='RoleMapping', index=False)
    
print("Permissions Loaded to File")

Permissions Loaded to File


In [243]:
AL_RbR = []
directory = 'Analysis\Approved_Access_12.2020\*'
for file in glob.glob(directory):
    domain = InferName(directory,file)
    ALDF = pd.read_excel(file, sheet_name=domain,na_values = '')
    ALDF['domain'] = domain
    AL = ALDF.values.tolist()
    for line in AL:
        if ',' in line[3]:
            roles = line[3].split(",")
            newroles = []
            for r in roles:
                newroles.append(r.lstrip())
        else:
            newroles = [line[3]]
        newline = [line[0],line[1],line[2],newroles,domain]
        AL_RbR.append(newline)

for line in IU:
    AL = [r[3] for r in AL_RbR if r[1]==line[-1]]
    line.append(AL)


print("Files Loaded")

file = r'Analysis\Approved_Access_12.2020\User_Implementation_TMS.xlsx'
AL = pd.read_excel(file, sheet_name='TMS',na_values = '').values.tolist()
AL_RbR = []
for line in AL:
    if ',' in line[3]:
        roles = line[3].split(",")
        newroles = []
        for r in roles:
            newroles.append(r.lstrip())
    else:
        newroles = [line[3]]
    newline = [line[0],line[1],line[2],newroles,line[4],line[5],line[6]]
    AL_RbR.append(newline)

print("Files Loaded")

Files Loaded


In [245]:
AL_RbR = []
directory = 'Analysis\Approved_Access_12.2020\*'
for file in glob.glob(directory):
    domain = InferName(directory,file)
    ALDF = pd.read_excel(file, sheet_name=domain,na_values = '')
    ALDF['domain'] = domain
    AL = ALDF.values.tolist()
    for line in AL:
        if ',' in line[3]:
            roles = line[3].split(",")
            newroles = []
            for r in roles:
                newroles.append(r.lstrip())
        else:
            newroles = [line[3]]
        newline = [line[0],line[1],line[2],newroles,domain]
        AL_RbR.append(newline)

for line in IU:
    AL = [r[3] for r in AL_RbR if r[1]==line[-1]]
    line.append(AL)


file = r'Analysis\Approved_Access_12.2020\User_Implementation_TMS.xlsx'
AL = pd.read_excel(file, sheet_name='TMS',na_values = '').values.tolist()
AL_RbR = []
for line in AL:
    if ',' in line[3]:
        roles = line[3].split(",")
        newroles = []
        for r in roles:
            newroles.append(r.lstrip())
    else:
        newroles = [line[3]]
    newline = [line[0],line[1],line[2],newroles,line[4],line[5],line[6]]
    AL_RbR.append(newline)


TMS_Access = ['Field (ODV)','Database (SCADA)','Integration','TGM Supervisor','State Estimator(TNA)','Planning','DOC Dispatcher'
              'DOC Supervisor','Communications (RTU)','Support','TGM Dispatcher','TGO Supervisor']

# {Value to sub: value in list}
siemenssubs = {
        "|USR|Database_(SCADA)": "Database (SCADA)",
        "|USR|Integration": "Integration",
        "|USR|State_Estimator(TNA)": "State Estimator(TNA)",
        "|USR|DOC_Operator": "DOC Dispatcher",
        "|USR|Communications_(RTU)": "Communications (RTU)",
        "|USR|TGM_Operator": "TGM Dispatcher",
        "N/A": "Planning",
        "N/A": "DOC Supervisor",
        "N/A": "Support",
        "N/A": "TGO Supervisor",
        "N/A": "Field (ODV)"
}

dec2020subs = {
        "|USR|Database_(SCADA)": "Database (SCADA)",
        "|USR|Integration": "Integration",
        "|USR|DOC_Operator": "DOC Operator",
        "|USR|Communications_(RTU)": "Communications (RTU)",
        "|USR|TGM_Operator": "TGM Operator",
        "|USR|Settings_Developer": 'Settings Developer'
}
#RoleMapping = pd.DataFrame(list(siemenssubs.items()),columns=['OSI_Role','Siemens_Role'])
#rev_subs = { v:k for k,v in subs.items()}



IUV = []
for u in IU:
    for r in u[4]:
        siemensrole = siemenssubs.get(r,r)
        newline = [item for item in u[0:4]] + [u[5],r]
        if siemensrole in u[6][0]:
            newline.append(siemensrole)
            newline.append("Success")
        else:
            newline.append("Access Not Found")
            newline.append("Failure")
        approvedrole = dec2020subs.get(r,r)
        try:
            if approvedrole in u[7][0]:
                newline.append(approvedrole)
                newline.append("Success")
            else:
                newline.append("Access Not Found")
                newline.append("Failure")
        except:
            newline.append("Not Originally Approved for Access")
            newline.append("Failure")
        if newline[7] == "Success" and newline[9] == "Success":
            newline.append("None")
        elif newline[9] == "Success" and newline[7] != "Success":
            newline.append("True-up PAM")
        else:
            newline.append("Review")
        IUV.append(newline)
        
print(IU[0])

        
with pd.ExcelWriter('Reports/10.IdentifiedUsers.xlsx') as writer:
    IUDF = pd.DataFrame(IU,columns=['Type','Name','UserName','Admin?','OSIAccess','RACF','CIPAccess','Dec2020ApprovedAccess'])
    IUDF.to_excel(writer, sheet_name='IdentifiedUsers',index=False)
    IUDF = pd.DataFrame(IUV,columns=['Type','Name','UserName','Admin?','RACF','OSIAccess','CIPAccess','VC1','Dec2020ApprovedAccess',"VC2","Recommended_Action"])
    IUDF.to_excel(writer, sheet_name='Validation',index=False)
    IUNCDF = pd.DataFrame(IUNC,columns=['Type','Name','UserName','Admin?','OSIAccess','RACF'])
    IUNCDF.to_excel(writer, sheet_name='NoCIP',index=False)
    NFEDDF = pd.DataFrame(NFED,columns=['Type','Name','UserName','Admin?','OSIAccess','RACF'])
    NFEDDF.to_excel(writer, sheet_name='NotFoundInEmpDirectory',index=False)
    RoleMapping = pd.DataFrame(list(siemenssubs.items()),columns=['OSI_Role','Siemens_Role'])
    RoleMapping.to_excel(writer, sheet_name='RoleMapping', index=False)
    
print("Permissions Loaded to File")

['User', 'James Melton', 'jmelton', 'Yes', ['|USR|Integration', '|USR|Administrator'], 'bdac1', 'TMS', 'LPHB', [['Shared User Access Listing Accounts', 'OTS Monarch - Non CIP', 'QAS Monarch - Non CIP', '19 - TGM Bldg C Vegas Conference Room', 'VPN Access - TGM OTS Users', 'Shared accounts on Windows servers', 'TMS CSIG Security VPN', 'ICD Manage', 'Oncor SubSite Administrator', '20 - BCC Control Room', '3 - TGM Telecom Room', '5 - EDOC Equipment Room', 'Technical Feasibility Exceptions', 'CIP Self-Assessment Members', 'Digitus Substation Access - System Admin', 'CIP SU SCADA Members', 'VPN Access - TMS Admins', 'CSG Restricted Procedures Members', 'VPN Access - TMS OSI Admins', '14 - EDOC CIP IDF Room', '21 - BCC Server Room', 'General User, NC: Editor', 'PDS Server - Non CIP', '18 - TGM Bldg C Equip Rm', 'AIX Access', 'CIP V5', 'ICD Approver L1-L2', 'VPN Access - TMS PDS Users', '4 - EDOC Dispatch Office', 'PDS Windows- Non CIP', 'QAS Windows - Non CIP', 'Secured Data Repository Admin

ValueError: 8 columns passed, passed data had 10 columns

In [ ]:
def GetCIPAccess(file):
    with open(file) as f:
        reader = csv.reader(f)
        S = [row for row in reader]
        R = list(set([item[0] for item in S]))
        access = list(set([item[2] for item in S]))
        mlist = []
        for r in R:
            nlist = list(set([item[1] for item in S if item[0] == r]))
            alist = list(set([item[2] for item in S if item[0] == r]))
            mlist.append([r,nlist,alist])
    return mlist, access
        
    
file = 'PAM\AllUser_Consolidated_CIPAccess_Report-202103080900.csv'
CIPA, access = GetCIPAccess(file)

file = 'uds/all_UDS/prod_uds.xml'
df, domain = GetUsers(file)
OSI = df.values.tolist()
OSI_U = [line for line in OSI if line[0]=='User']

file = 'Analysis/PAM Access Match.csv'
with open(file) as f:
    reader = csv.reader(f)
    Match = [row for row in reader]
    RACF = [m[1] for m in Match]
    UN = [m[0] for m in Match]
    
    
NFED = []
IU = []
IUNC = []
CIPR = [item[0] for item in CIPA]
for line in OSI_U:
    r = [m[1] for m in Match if m[0]==line[2]]
    line.append(r[0])
    if r[0] == 'Not found in employee directory':
        NFED.append(line)
    elif r[0] not in CIPR:
        IUNC.append(line)
    else:
        IU.append(line)
        
print("Users that Need to be Added to the Match File")
for line in OSI_U:
    r = [m[1] for m in Match if m[0]==line[2]]
    if len(r) == 0:
        print(line)
print("Validation Complete")

print("Users that have multiple entries in the Match File")
for line in OSI_U:
    r = [m[1] for m in Match if m[0]==line[2]]
    if len(r) > 1:
        print(line)
print("Validation Complete")



print("Identified Users:",len(IU))
print("Identified Users, Not CIP:",len(IUNC))
print("Not Found in Employee Directory:",len(NFED))

file = r'Analysis\Approved_Access_12.2020\User_Implementation_TMS.xlsx'
AL = pd.read_excel(file, sheet_name='TMS',na_values = '').values.tolist()
AL_RbR = []
for line in AL:
    if ',' in line[3]:
        roles = line[3].split(",")
        newroles = []
        for r in roles:
            newroles.append(r.lstrip())
    else:
        newroles = [line[3]]
    newline = [line[0],line[1],line[2],newroles,line[4],line[5],line[6]]
    AL_RbR.append(newline)
        



for line in IU:
    CA = [c[2] for c in CIPA if c[0]==line[-1]]
    AL = [r[3] for r in AL_RbR if r[1]==line[-1]]
    line.append(CA)
    line.append(AL)

print("Files Loaded")

In [ ]:
TMS_Access = ['Field (ODV)','Database (SCADA)','Integration','TGM Supervisor','State Estimator(TNA)','Planning','DOC Dispatcher'
              'DOC Supervisor','Communications (RTU)','Support','TGM Dispatcher','TGO Supervisor']

# {Value to sub: value in list}
siemenssubs = {
        "|USR|Database_(SCADA)": "Database (SCADA)",
        "|USR|Integration": "Integration",
        "|USR|State_Estimator(TNA)": "State Estimator(TNA)",
        "|USR|DOC_Operator": "DOC Dispatcher",
        "|USR|Communications_(RTU)": "Communications (RTU)",
        "|USR|TGM_Operator": "TGM Dispatcher",
        "N/A": "Planning",
        "N/A": "DOC Supervisor",
        "N/A": "Support",
        "N/A": "TGO Supervisor",
        "N/A": "Field (ODV)"
}

dec2020subs = {
        "|USR|Database_(SCADA)": "Database (SCADA)",
        "|USR|Integration": "Integration",
        "|USR|DOC_Operator": "DOC Operator",
        "|USR|Communications_(RTU)": "Communications (RTU)",
        "|USR|TGM_Operator": "TGM Operator",
        "|USR|Settings_Developer": 'Settings Developer'
}
#RoleMapping = pd.DataFrame(list(siemenssubs.items()),columns=['OSI_Role','Siemens_Role'])
#rev_subs = { v:k for k,v in subs.items()}



IUV = []
for u in IU:
    for r in u[4]:
        siemensrole = siemenssubs.get(r,r)
        newline = [item for item in u[0:4]] + [u[5],r]
        if siemensrole in u[6][0]:
            newline.append(siemensrole)
            newline.append("Success")
        else:
            newline.append("Access Not Found")
            newline.append("Failure")
        approvedrole = dec2020subs.get(r,r)
        try:
            if approvedrole in u[7][0]:
                newline.append(approvedrole)
                newline.append("Success")
            else:
                newline.append("Access Not Found")
                newline.append("Failure")
        except:
            newline.append("Not Originally Approved for Access")
            newline.append("Failure")
        if newline[7] == "Success" and newline[9] == "Success":
            newline.append("None")
        elif newline[9] == "Success" and newline[7] != "Success":
            newline.append("True-up PAM")
        else:
            newline.append("Review")
        IUV.append(newline)
        


        
with pd.ExcelWriter('Reports/10.IdentifiedUsers.xlsx') as writer:
    IUDF = pd.DataFrame(IU,columns=['Type','Name','UserName','Admin?','OSIAccess','RACF','CIPAccess','Dec2020ApprovedAccess'])
    IUDF.to_excel(writer, sheet_name='IdentifiedUsers',index=False)
    IUDF = pd.DataFrame(IUV,columns=['Type','Name','UserName','Admin?','RACF','OSIAccess','CIPAccess','VC1','Dec2020ApprovedAccess',"VC2","Recommended_Action"])
    IUDF.to_excel(writer, sheet_name='Validation',index=False)
    IUNCDF = pd.DataFrame(IUNC,columns=['Type','Name','UserName','Admin?','OSIAccess','RACF'])
    IUNCDF.to_excel(writer, sheet_name='NoCIP',index=False)
    NFEDDF = pd.DataFrame(NFED,columns=['Type','Name','UserName','Admin?','OSIAccess','RACF'])
    NFEDDF.to_excel(writer, sheet_name='NotFoundInEmpDirectory',index=False)
    RoleMapping = pd.DataFrame(list(siemenssubs.items()),columns=['OSI_Role','Siemens_Role'])
    RoleMapping.to_excel(writer, sheet_name='RoleMapping', index=False)
    
print("Permissions Loaded to File")

In [ ]:
print("Process Complete")